In [1]:
# modified from https://github.com/pytorch/examples/blob/master/imagenet/main.py
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
from utils import *

/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
#imports
import pandas as pd
import numpy as np
# from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# df_unsw_normal = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/dataset/train/unsw_normal.csv')
df_bot_dos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/datasetv2/train/bot_iot_dos.csv')
df_bot_ddos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/datasetv2/train/bot_iot_ddos.csv')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/tmp/ipykernel_117611/2989520626.py:13: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bot_dos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/datasetv2/train/bot_iot_dos.csv')
/tmp/ipykernel_117611/2989520626.py:14: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bot_ddos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/datasetv2/train/bot_iot_ddos.csv')


In [3]:
df_combined = pd.concat([df_bot_dos, df_bot_ddos], ignore_index=True)
df_combined['sport'] = np.where(df_combined['sport'] == '0x0303', 0x0303, df_combined['sport'])
df_combined['sport'] = np.where(df_combined['sport'] == '0x5000', 0x5000, df_combined['sport'])
df_combined = df_combined[(df_combined['state']=='INT') | (df_combined['state']=='REQ')| (df_combined['state']=='RST')| (df_combined['state']=='ACC')| (df_combined['state']=='CON')| (df_combined['state']=='FIN')]
df_combined['flgs'] = df_combined['flgs'].map({'e': 0, 'e s': 1, 'e g': 2, 'eU': 3, 'e *':4, 'e d':5})
df_combined['state'] = df_combined['state'].map({'INT': 0, 'REQ': 1, 'RST': 2, 'ACC': 3, 'CON':4, 'FIN':6})

x = df_combined.loc[:, df_combined.columns != 'category']
x_ = np.zeros((x.shape[0], 49))
x_[:, :43] = x
x = x_

# x = (x-x.min())/(x.max()-x.min())

df_combined.category = pd.factorize(df_combined.category)[0]
y = df_combined['category']

scaler = StandardScaler()
x = scaler.fit_transform(x)

# for i in [0,1,2,3,7,8]:
#     x[:,i] = np.where(x[:,i] > 0, np.log(x[:,i]), 0)
x = x.reshape(x.shape[0], 1, 7, 7)    
x_label_data = torch.tensor(x, dtype=torch.float).to(device)
y_label_data = torch.tensor(y, dtype=torch.long).to(device)

dataset = TensorDataset(x_label_data, y_label_data)
train_size = int(0.7 * len(dataset))
valid_size = int(0.25 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# Create data loaders
batch_size = 8192
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
quant = 'TNN'
class Netb(nn.Module):
    def __init__(self):
        super(Netb, self).__init__()
        # self.fc0 = QLinear(7 * 7, 64 * 64, bias=False)
        self.conv1 = QConv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1, bias=False, quant=quant)
        self.conv2 = QConv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, bias=False, quant=quant)
        self.linear1 = QLinear(64 * 7 * 7, 128, bias=False, quant=quant)
        self.linear2 = QLinear(128, 3, bias=False, quant=quant)
        self.act4 = nn.Softmax(dim = 1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_b = Netb().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_b.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 2
for epoch in range(num_epochs):
    model_b.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_b(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_b.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_b(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_b.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_b(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_b.state_dict(), 'saved_best_model_bcnn.pt')

Epoch [1/2], Validation Accuracy: 50.00%4
Epoch [2/2], Validation Accuracy: 100.00%
Test Accuracy: 100.00%


In [1]:
class Netb(nn.Module):
    def __init__(self):
        super(Netb, self).__init__()
        # self.fc0 = QLinear(7 * 7, 64 * 64, bias=False)
        self.conv1 = QConv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1, bias=False, quant=quant)
        self.conv2 = QConv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, bias=False, quant=quant)
        self.linear1 = QLinear(64 * 7 * 7, 128, bias=False, quant=quant)
        self.linear2 = QLinear(128, 3, bias=False, quant=quant)
        self.act4 = nn.Softmax(dim = 1)
    def forward(self, x):
        x1 = self.conv1(x)
        x = self.conv2(x1)
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = self.linear2(x)
        
        x = self.act4(x)
        return x1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_b = Netb().to(device)
model_b.load_state_dict(torch.load('saved_best_model_bcnn.pt'))
save_model_parameters_to_txt(model_b, 'tcnn_model_parameters.txt')

NameError: name 'save_model_parameters_to_txt' is not defined

In [101]:
class Ternary(torch.nn.Module):
    def __init__(self):
        super(Ternary, self).__init__()
        config = {}
        self.config = config
        self.delta = config.setdefault("delta", 0.5)
        self.momentum = config.setdefault("momentum", 0.01)
        self.track_running_stats = config.setdefault("track_running_stats", True)
        self.order = config.setdefault('order', 2)
        # self.use_scale = config.setdefault('use_scale', True)
        assert self.momentum <= 1 and self.order > 0 and self.delta > 0
        self.register_buffer("running_delta", torch.zeros(1))
        self.reset_parameters()

    def reset_parameters(self):
        if self.momentum > 0:
            self.running_delta.fill_(self.delta * 0.7979)
        else:
            self.running_delta.fill_(self.delta)

    def forward(self, input_f):
        return ternary(input_f, self.running_delta, self.delta, self.momentum,
                       self.training and self.track_running_stats, self.order)

    def extra_repr(self):
        return ", ".join(["{}={}".format(k, v) for k, v in self.config.items()])

class QConv2d(torch.nn.Conv2d):
    qa_config = {}
    qw_config = {}
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False,
                 padding_mode='zeros', quant = 'BNN'):
        super().__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias, padding_mode)
        
        if quant == 'BNN':
            self.input_quantizer = BinaryActivation()
            self.weight_quantizer = BinaryActivation()
        elif quant == 'TNN':
            self.input_quantizer = Ternary()
            self.weight_quantizer = Ternary()
        elif quant == 'TBN':
            self.input_quantizer = Ternary()
            self.weight_quantizer = BinaryActivation()
        self.i = 0

    def forward(self, input_f):
        input_t = self.input_quantizer(input_f)
        weight_b = self.weight_quantizer(self.weight)
        out = F.conv2d(input_t, weight_b, self.bias, self.stride, self.padding, self.dilation, self.groups)
        if(self.i == 0):
            self.i = 1
            # print(input_t)
            print(self.weight)
            print(weight_b)

        return out

In [102]:
class Netb(nn.Module):
    def __init__(self):
        super(Netb, self).__init__()
        # self.fc0 = QLinear(7 * 7, 64 * 64, bias=False)
        self.conv1 = QConv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1, bias=False, quant=quant)
        self.conv2 = QConv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, bias=False, quant=quant)
        self.linear1 = QLinear(64 * 7 * 7, 128, bias=False, quant=quant)
        self.linear2 = QLinear(128, 3, bias=False, quant=quant)
        self.act4 = nn.Softmax(dim = 1)
    def forward(self, x):
        print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
        x1 = self.conv1(x)
        print("BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB")
        print(x1)
        x = self.conv2(x1)
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = self.linear2(x)
        
        x = self.act4(x)
        return x1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_b = Netb().to(device)
model_b.load_state_dict(torch.load('saved_best_model_bcnn.pt'))

<All keys matched successfully>

In [126]:
nn = name.split('.')[0]
getattr(getattr(model_b, nn).weight_quantizer, 'forward')(kernel)

tensor([[[[ 1.,  1.,  1.],
          [ 0., -1., -1.],
          [ 1.,  1.,  1.]]]], device='cuda:0')

In [124]:
nn = name.split('.')[0]

'linear2'

In [123]:
for name, param in model_b.named_parameters():
    print(name)

conv1.weight
conv2.weight
linear1.weight
linear2.weight


In [103]:
a = model_b(input)

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Parameter containing:
tensor([[[[ 2.2384e-01,  2.2946e-01,  2.7293e-01],
          [-9.5651e-04, -2.6885e-01, -2.5753e-01],
          [ 1.5774e-01,  1.4375e-01,  6.2775e-01]]],


        [[[ 5.7201e-01, -3.0356e-01,  2.7134e-01],
          [-4.3390e-02,  1.4793e-01,  1.4906e-01],
          [ 4.0869e-01, -1.7056e-01, -2.2095e-01]]],


        [[[ 4.4437e-01, -1.8334e-01, -2.1462e-01],
          [ 2.9422e-01,  4.7742e-01,  8.8640e-01],
          [-2.5724e-01,  2.4763e-01, -1.9743e-04]]],


        [[[-3.3729e-01, -3.0430e-01, -3.1893e-01],
          [-4.3160e-01,  2.2575e-01, -5.6895e-01],
          [-4.7998e-02, -2.6802e-01, -8.5414e-01]]],


        [[[-9.3040e-01, -2.5569e-01,  1.6291e-01],
          [ 1.0130e-01, -1.9798e-01, -2.3457e-01],
          [ 4.7315e-02,  2.8065e-01, -7.1974e-02]]],


        [[[ 1.2168e-01,  2.4957e-01, -1.7423e-01],
          [-2.8821e-01,  8.3267e-02, -3.2709e-01],
          [ 2.8243e-01,  1.2911e-01, -2.6657e-01]]],


     

In [77]:
for name, param in model_b.named_parameters():
    if(name == 'conv1.weight'):
        x = param

In [81]:
_ternary(x[0],0.1440)

tensor([[[ 1.,  1.,  1.],
         [ 0., -1., -1.],
         [ 1.,  0.,  1.]]], device='cuda:0')

In [48]:
def _ternary(x: torch.Tensor, delta: float):
    return (x >= delta).float() - (x <= -delta).float()
_ternary(tensor,0.1440)

tensor([[ 1.,  1.,  1.],
        [ 0., -1., -1.],
        [ 1.,  0.,  1.]])

In [50]:
_ternary(input,0.2969)

tensor([[[[ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.]]]], device='cuda:0')

In [96]:
image = torch.tensor([[[[ 1., -1.,  1., -1.,  1., -1.,  1.],
                        [ 1., -1.,  1., -1.,  1., -1.,  1.],
                        [ 1., -1.,  1., -1.,  1., -1.,  1.],
                        [ 1., -1.,  1., -1.,  1., -1.,  1.],
                        [ 1., -1.,  1., -1.,  1., -1.,  1.],
                        [ 1., -1.,  1., -1.,  1., -1.,  1.],
                        [ 1., -1.,  1., -1.,  1., -1.,  1.]]]], device='cuda:0')

kernel = torch.tensor([[ 2.2384e-01,  2.2946e-01,  2.7293e-01],
                        [-9.5651e-04, -2.6885e-01, -2.5753e-01],
                        [ 1.5774e-01,  1.4375e-01,  6.2775e-01]], device='cuda:0')

# Thêm chiều cho kernel để phù hợp với yêu cầu của hàm conv2d
kernel = kernel.unsqueeze(0).unsqueeze(0)

# Thực hiện tích chập
output = F.conv2d(_ternary(image,0.2969), _ternary(kernel,0.5), padding=1)

print(output)

tensor([[[[-1.,  1., -1.,  1., -1.,  1.,  0.],
          [-1.,  1., -1.,  1., -1.,  1.,  0.],
          [-1.,  1., -1.,  1., -1.,  1.,  0.],
          [-1.,  1., -1.,  1., -1.,  1.,  0.],
          [-1.,  1., -1.,  1., -1.,  1.,  0.],
          [-1.,  1., -1.,  1., -1.,  1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]]], device='cuda:0')


In [97]:
_ternary(kernel,0.5)

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 1.]]]], device='cuda:0')

In [53]:
_ternary(output,0.1440)

tensor([[[[-1.,  1., -1.,  1., -1.,  1., -1.],
          [-1.,  1., -1.,  1., -1.,  1., -1.],
          [-1.,  1., -1.,  1., -1.,  1., -1.],
          [-1.,  1., -1.,  1., -1.,  1., -1.],
          [-1.,  1., -1.,  1., -1.,  1., -1.],
          [-1.,  1., -1.,  1., -1.,  1., -1.],
          [ 0.,  1., -1.,  1., -1.,  1., -1.]]]], device='cuda:0')

In [34]:
model_b.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[ 2.2384e-01,  2.2946e-01,  2.7293e-01],
                        [-9.5651e-04, -2.6885e-01, -2.5753e-01],
                        [ 1.5774e-01,  1.4375e-01,  6.2775e-01]]],
              
              
                      [[[ 5.7201e-01, -3.0356e-01,  2.7134e-01],
                        [-4.3390e-02,  1.4793e-01,  1.4906e-01],
                        [ 4.0869e-01, -1.7056e-01, -2.2095e-01]]],
              
              
                      [[[ 4.4437e-01, -1.8334e-01, -2.1462e-01],
                        [ 2.9422e-01,  4.7742e-01,  8.8640e-01],
                        [-2.5724e-01,  2.4763e-01, -1.9743e-04]]],
              
              
                      [[[-3.3729e-01, -3.0430e-01, -3.1893e-01],
                        [-4.3160e-01,  2.2575e-01, -5.6895e-01],
                        [-4.7998e-02, -2.6802e-01, -8.5414e-01]]],
              
              
                      [[[-9.3040e-01, -2.5569e-01,  1.6291e-01

In [67]:
a[0][0]

tensor([[ 0.,  1., -1.,  1., -1.,  1., -1.],
        [ 0.,  2., -2.,  2., -2.,  2., -1.],
        [ 0.,  2., -2.,  2., -2.,  2., -1.],
        [ 0.,  2., -2.,  2., -2.,  2., -1.],
        [ 0.,  2., -2.,  2., -2.,  2., -1.],
        [ 0.,  2., -2.,  2., -2.,  2., -1.],
        [ 0.,  1., -1.,  1., -1.,  1., -1.]], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [59]:
a = model_b(input)

tensor([[[[ 0.,  1., -1.,  ..., -1.,  1., -1.],
          [ 0.,  2., -2.,  ..., -2.,  2., -1.],
          [ 0.,  2., -2.,  ..., -2.,  2., -1.],
          ...,
          [ 0.,  2., -2.,  ..., -2.,  2., -1.],
          [ 0.,  2., -2.,  ..., -2.,  2., -1.],
          [ 0.,  1., -1.,  ..., -1.,  1., -1.]],

         [[ 0.,  1., -1.,  ..., -1.,  1., -1.],
          [-2.,  4., -4.,  ..., -4.,  4., -3.],
          [-2.,  4., -4.,  ..., -4.,  4., -3.],
          ...,
          [-2.,  4., -4.,  ..., -4.,  4., -3.],
          [-2.,  4., -4.,  ..., -4.,  4., -3.],
          [-2.,  3., -3.,  ..., -3.,  3., -1.]],

         [[ 1., -1.,  1.,  ...,  1., -1.,  2.],
          [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  2., -2.,  ..., -2.,  2., -2.]],

         ...,

         [[-3.,  3., -3.,  ..., -3.,  3., -1.],
          [-1.,  0.,  

In [20]:
input = torch.zeros(1,1,7,7).to(device)
for i in range(7):
    for j in range(7):
        input[0][0][i][j] = np.power(-1,j)
input.to(device)


tensor([[[[ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.],
          [ 1., -1.,  1., -1.,  1., -1.,  1.]]]], device='cuda:0')

In [229]:
def convert_and_concatenate(arr):
    # Chuyển đổi -1 thành 1 và 1 thành 0
    arr = torch.flip(arr, dims=[0])
    converted_arr = [1 if x == -1 else 0 for x in arr]
    # Ghép các phần tử lại thành một chuỗi
    concatenated_str = ''.join(map(str, converted_arr))

    current_length = len(concatenated_str)
    total_length = (np.ceil(current_length/32)*32).astype(int)
    if current_length < total_length:
        padding_length = int(total_length - current_length)
        concatenated_str = '0' * padding_length + concatenated_str
    # Chuyển chuỗi thành số nhị phân và sau đó thành số hex
    hex_value = hex(int(concatenated_str, 2))[2:]  # Bỏ tiền tố '0x'

    # Đảm bảo chuỗi hex đủ độ dài cần thiết
    hex_length = total_length // 4  # 1 hex digit = 4 bits
    if len(hex_value) < hex_length:
        hex_value = '0' * (hex_length - len(hex_value)) + hex_value
    return hex_value

def to_hex(x):
    return f"0x{x:08x}"

def save_model_parameters_to_txt(model, file_path):
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            layer_name = name.split('.')[0]
            if 'conv' in name:
                f.write(f'{name}\n')
                f.write(f'layer_name: {layer_name}\n')
                f.write(f'input_channel: {param.shape[1]}\n')
                f.write(f'output_channel: {param.shape[0]}\n')
                f.write(f'kernel_size: 3\n')
                f.write(f'stride: 1\n')
                f.write(f'padding: 1\n')
                f.write(f'dilation: 1\n')
                f.write(f'quant_type: 0\n')
                f.write(f'input_thres: 0\n')
                for param_e in param:
                    sichannel = int(np.ceil(param_e.shape[0]/32))
                    # param_q = torch.zeros([sichannel,param_e.shape[1],param_e.shape[2]])
                    # param_q = param_q.int()
                    for c in range(sichannel):
                        for i in range(param_e.shape[1]):
                            for j in range(param_e.shape[2]):
                                hex_conv = convert_and_concatenate(_binary(param_e[sichannel*0:sichannel*32, i, j]))
                                if j == param_e.shape[2]-1:
                                    f.write(f'0x{hex_conv}')
                                else:
                                    f.write(f'0x{hex_conv}, ')
                            f.write(f'\n')
                        f.write(f'\n')
            if 'fc' in name or 'linear' in name:
                f.write(f'{name}\n')
                f.write(f'layer_name: {layer_name}\n')
                f.write(f'input_channel: {param.shape[1]}\n')
                f.write(f'output_channel: {param.shape[0]}\n')
                f.write(f'quant_type: 0\n')
                f.write(f'input_thres: 0.0\n')
                param = _binary(param)
                for param_e in param:
                    hex_conv = convert_and_concatenate(param_e)
                    segments = [hex_conv[i:i+8] for i in range(0, len(hex_conv), 8)]
                    segments.reverse()
                    for segment in segments:
                        f.write(f'0x{segment}\n')
                f.write(f'\n')
        return 0

# Gọi hàm để lưu thông số mô hình
a = save_model_parameters_to_txt(model_b, 'bcnn_model_parameters.txt')

In [105]:
i = 0
for name, param in model_b.named_parameters():
    if 'conv' in name:
        for param_e in param:
            sichannel = int(np.ceil(param_e.shape[0]/32))
            param_q = torch.zeros([sichannel,param_e.shape[1],param_e.shape[2]])
            for c in range(sichannel):
                for i in range(param_e.shape[1]):
                    for j in range(param_e.shape[2]):
                        param_q[c,i,j] = int(convert_and_concatenate(_binary(param_e[sichannel*0:sichannel*32, i, j])),16)
            param_q = param_q.int()
            # for i in param_q:
            #     print(i)
            # print(hex(param_q))
            # print(f"Giá trị hex: {hex(param_q)}")
            # print(hex(param_q))
        print(param.shape)
        # break

torch.Size([32, 1, 3, 3])
torch.Size([64, 32, 3, 3])


# **BNN**

In [338]:
class BinaryActivationFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Binarize input to -1 or 1
        output = input.sign()
        output[output==0] = 1
        ctx.save_for_backward(input)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input.abs() > 1] = 0
        return grad_input

class BinaryActivation(nn.Module):
    def forward(self, input):
        return BinaryActivationFunction.apply(input)

class QLinear(torch.nn.Linear):
    qa_config = {}
    qw_config = {}

    def __init__(self, in_features, out_features, bias=False):
        super().__init__(in_features, out_features, bias)
        # self.input_quantizer = BinaryActivation()
        self.weight_quantizer = BinaryActivation()

    def forward(self, input_f):
        # input_t = self.input_quantizer(input_f)
        weight_b = self.weight_quantizer(self.weight)
        out = F.linear(input_f, weight_b, self.bias)
        return out

In [339]:
class Netb(nn.Module):
    def __init__(self):
        super(Netb, self).__init__()
        self.input_quantizer = BinaryActivation()

        self.linear1 = QLinear(13, 128, bias = False)
        self.bn1 = nn.BatchNorm1d(128)
        # self.i1 = nn.Identity()

        self.linear2 = QLinear(128, 128, bias = False)
        self.bn2 = nn.BatchNorm1d(128)
        # self.i2 = nn.Identity()

        self.linear3 = QLinear(128, 128, bias = False)
        self.bn3 = nn.BatchNorm1d(128)
        # self.i3 = nn.Identity()

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.input_quantizer(x)
        x = self.linear1(x)

        x = self.input_quantizer(x)
        x = self.linear2(x)
        # x = self.bn2(x)
        # x = self.i2(x)
        x = self.input_quantizer(x)
        x = self.linear3(x)
        # x = self.bn3(x)
        # x = self.i3(x)
        x = self.input_quantizer(x)
        x = self.linear4(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_b = Netb().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_b.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 5
for epoch in range(num_epochs):
    model_b.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_b(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_b.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_b(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_b.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_b(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_b.state_dict(), 'saved_best_model_bnn.pt')

Epoch [1/5], Validation Accuracy: 91.39%6
Epoch [2/5], Validation Accuracy: 89.58%6
Epoch [3/5], Validation Accuracy: 92.36%8
Epoch [4/5], Validation Accuracy: 92.36%5
Epoch [5/5], Validation Accuracy: 92.36%4
Test Accuracy: 92.25%


In [729]:
class BinaryActivationFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Binarize input to -1 or 1
        output = input.sign()
        output[output==0] = 1
        ctx.save_for_backward(input)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input.abs() > 1] = 0
        return grad_input

class BinaryActivation(nn.Module):
    def forward(self, input):
        return BinaryActivationFunction.apply(input)

class QLinear(torch.nn.Linear):
    qa_config = {}
    qw_config = {}
    def __init__(self, in_features, out_features, bias=True):
        super().__init__(in_features, out_features, bias)
        self.weight_quantizer = BinaryActivation()
    def forward(self, input_f):
        # weight_b = self.weight_quantizer(self.weight)
        out = F.linear(input_f, self.weight, self.bias)
        return out

class Netb(nn.Module):
    def __init__(self):
        super(Netb, self).__init__()
        self.input_quantizer = BinaryActivation()
        self.linear1 = QLinear(13, 128, bias = False)

        self.linear2 = QLinear(128, 128, bias = False)

        self.linear3 = QLinear(128, 128, bias = False)

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.input_quantizer(x)
        x = self.linear1(x)
        print("linear1")
        print(x)

        x = self.input_quantizer(x)
        x = self.linear2(x)
        print("linear2")
        print(x)

        x = self.input_quantizer(x)
        x = self.linear3(x)
        print("linear3")
        print(x)
        
        x = self.input_quantizer(x)
        x = self.linear4(x)
        print("linear4")
        print(x)
        x = self.act4(x)
        print(x)
        return x

model_bnn = Netb().to(device)
model_bnn.linear1.state_dict()['weight'].copy_(model_b.linear1.state_dict()['weight'].sign())
model_bnn.linear2.state_dict()['weight'].copy_(model_b.linear2.state_dict()['weight'].sign())
model_bnn.linear3.state_dict()['weight'].copy_(model_b.linear3.state_dict()['weight'].sign())
model_bnn.linear4.state_dict()['weight'].copy_(model_b.linear4.state_dict()['weight'].sign())

tensor([[ 1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,
          1.,  1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,
         -1.,  1., -1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,
         -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
         -1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,
         -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,
         -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,
          1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,
         -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,
          1., -1.],
        [-1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
         -1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,
         -1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,
         -1.,  1.,  1.,  1.,  1.

In [730]:
model_bnn.eval()
correct = 0
total = 0
# images = torch.tensor([[0.0000,  0.0000,  0.0000,  0.0000, -0.6556,  0.9844,  0.2236, -1.4968, -1.4968, -0.9711, -0.1279,  0.2186,  0.2208]]).to(device)
images = torch.tensor([[ 0.,          0.,          0.,          0.,         -0.48889792, -1.0158767,
  0.2236068,  -1.4960896,  -1.4960909,   0.74484456, -0.1279424,   0.21856718,
  0.22078077]]).to(device)
with torch.no_grad():
    outputs = model_bnn(images)


linear1
tensor([[  5.,   7.,   7.,   1.,  -3.,  -3.,  -7.,   3.,  -1.,  -7.,  -3.,   3.,
          -5.,   1.,   1.,   5.,  -5.,   1.,  -7.,   5.,   1.,   1.,  -1.,   1.,
          -3.,   5.,  -3.,   3.,   1.,   5.,  -3.,   3.,   1.,  -3.,   5.,  -3.,
          -3.,  -1.,   9.,  -1.,  -5.,   3.,   1.,   3.,   9.,   3.,   5.,  -7.,
          -1.,   5.,   5.,  -3.,   3.,   5.,  -3.,   7.,  -3.,   5.,   3.,  -7.,
           5.,  -1.,  -5.,  -3.,  -3.,   3.,  -7.,  -3.,  -3.,  -3.,   1.,   5.,
           3.,   1.,  -1.,   1.,  -3.,  -1.,   3.,   1.,   1.,   3.,   1.,  -1.,
           7.,  -3.,  -5.,  -5.,  -7.,  -3.,  -5.,  -1.,  -3.,  -5.,   1.,  -9.,
          -3.,  -5.,   5.,  -3., -11.,   7.,  -1.,   3.,  -5.,   3.,  -3.,  -1.,
           5.,  -5.,  -3.,  -7.,  -1.,  -5.,   5.,   1.,  -1.,   3.,  -7.,   5.,
           3.,  -5.,  -5.,   5.,   7.,  -5.,  -3.,  -5.]], device='cuda:0')
linear2
tensor([[-24., -16., -14., -32., -32.,  20.,  -4., -16., -10.,  -8.,  10., -26.,
         -22., -1

In [341]:
model_bnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_bnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')
# torch.save(model_bnn.state_dict(), 'saved_best_model_bnn_1.pt')

Test Accuracy: 92.25%


In [272]:
def convert_and_concatenate(arr):
    # Chuyển đổi -1 thành 1 và 1 thành 0
    arr = torch.flip(arr, dims=[0])
    converted_arr = [1 if x == -1 else 0 for x in arr]
    # Ghép các phần tử lại thành một chuỗi
    concatenated_str = ''.join(map(str, converted_arr))

    current_length = len(concatenated_str)
    total_length = (np.ceil(current_length/32)*32).astype(int)
    if current_length < total_length:
        padding_length = int(total_length - current_length)
        concatenated_str = '0' * padding_length + concatenated_str
    # Chuyển chuỗi thành số nhị phân và sau đó thành số hex
    hex_value = hex(int(concatenated_str, 2))[2:]  # Bỏ tiền tố '0x'

    # Đảm bảo chuỗi hex đủ độ dài cần thiết
    hex_length = total_length // 4  # 1 hex digit = 4 bits
    if len(hex_value) < hex_length:
        hex_value = '0' * (hex_length - len(hex_value)) + hex_value
    return hex_value

def save_model_parameters_to_txt(model, file_path):
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f'{name}\n')
            f.write(f'input_channel: {param.shape[1]}\n')
            f.write(f'output_channel: {param.shape[0]}\n')
            for param_e in param:
                hex_conv = convert_and_concatenate(param_e)
                segments = [hex_conv[i:i+8] for i in range(0, len(hex_conv), 8)]
                segments.reverse()
                for segment in segments:
                    f.write(f'0x{segment}\n')
            f.write(f'\n')
    return 0

# Gọi hàm để lưu thông số mô hình
a = save_model_parameters_to_txt(model_bnn, 'bnn_model_parameters.txt')

# **FP**

In [302]:
class Net_fp(nn.Module):
    def __init__(self):
        super(Net_fp, self).__init__()
        self.linear1 = nn.Linear(13, 128, bias = False)
        # self.bn1 = nn.BatchNorm1d(128)
        self.act1 = nn.ReLU()

        self.linear2 = nn.Linear(128, 128, bias = False)
        # self.bn2 = nn.BatchNorm1d(128)
        self.act2 = nn.ReLU()


        self.linear3 = nn.Linear(128, 128, bias = False)
        # self.bn3 = nn.BatchNorm1d(128)
        self.act3 = nn.ReLU()

        self.linear4 = nn.Linear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):

        x = self.linear1(x)
        # x = self.bn1(x)
        x = self.act1(x)

        x = self.linear2(x)
        # x = self.bn2(x)
        x = self.act2(x)

        x = self.linear3(x)
        # x = self.bn3(x)
        x = self.act3(x)

        x = self.linear4(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_fp = Net_fp().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_fp.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 2
for epoch in range(num_epochs):
    model_fp.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_fp(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_fp.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_fp(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_fp.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_fp(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_fp.state_dict(), 'saved_best_model_fp.pt')

Epoch [1/2], Validation Accuracy: 91.52%1
Epoch [2/2], Validation Accuracy: 91.63%0
Test Accuracy: 91.51%


In [303]:
def save_model_fp_parameters_to_txt(model, file_path):
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f'{name}\n')
            if('linear' in name and 'weight'in name):
                f.write(f'input_channel: {param.shape[1]}\n')
                f.write(f'output_channel: {param.shape[0]}\n')
            f.write(f'{param}\n')
            f.write(f'\n')
    return 0

# Gọi hàm để lưu thông số mô hình
a = save_model_fp_parameters_to_txt(model_fp, 'fp_model_parameters.txt')

# **TNN**

In [623]:
def _ternary(x: torch.Tensor, delta: float):
    return (x >= delta).float() - (x <= -delta).float()

class _ternary_py(torch.autograd.Function):
    @staticmethod
    def ternary_backward(grad_output: torch.Tensor, x: torch.Tensor, delta: float, order: int, threshold: float):
        scale = 2 * delta
        assert threshold <= scale
        tmp = torch.zeros_like(grad_output)
        tmp += ((x >= -threshold) & (x <= threshold)).float() * order * \
               (torch.fmod(x / delta + 3, 2) - 1).abs().pow(order - 1)
        return grad_output * tmp

    @staticmethod
    def forward(ctx, *inputs) -> torch.Tensor:
        input_f, running_delta, delta, momentum, training, ctx.order = inputs
        if momentum > 0:
            if training:
                ctx.delta = input_f.norm(1).item() * (delta / input_f.numel())  # = delta * |input_f|_1 / n
                running_delta.data = momentum * ctx.delta + (1.0 - momentum) * running_delta.data
            else:
                ctx.delta = running_delta.data.item()
        else:
            ctx.delta = delta
        # input_t = _ternary(input_f, ctx.delta) * (2 * ctx.delta)
        input_t = _ternary(input_f, ctx.delta)
        ctx.save_for_backward(input_f)
        return input_t

    @staticmethod
    def backward(ctx, *grad_outputs):
        grad_output, = grad_outputs
        input_f, = ctx.saved_tensors
        grad_input = _ternary_py.ternary_backward(grad_output, input_f, ctx.delta, ctx.order, 2. * ctx.delta)
        return grad_input, None, None, None, None, None, None, None, None, None


def ternary(input_f: torch.Tensor, running_delta, delta, momentum, training, order):
    return _ternary_py.apply(input_f, running_delta, delta, momentum, training, order)


class Ternary(torch.nn.Module):
    def __init__(self):
        super(Ternary, self).__init__()
        config = {}
        self.config = config
        self.delta = config.setdefault("delta", 0.5)
        self.momentum = config.setdefault("momentum", 0.01)
        self.track_running_stats = config.setdefault("track_running_stats", True)
        self.order = config.setdefault('order', 2)
        # self.use_scale = config.setdefault('use_scale', True)
        assert self.momentum <= 1 and self.order > 0 and self.delta > 0
        self.register_buffer("running_delta", torch.zeros(1))
        self.reset_parameters()

    def reset_parameters(self):
        if self.momentum > 0:
            self.running_delta.fill_(self.delta * 0.7979)
        else:
            self.running_delta.fill_(self.delta)

    def forward(self, input_f):
        return ternary(input_f, self.running_delta, self.delta, self.momentum,
                       self.training and self.track_running_stats, self.order)

    def extra_repr(self):
        return ", ".join(["{}={}".format(k, v) for k, v in self.config.items()])
class QLinear(torch.nn.Linear):
    def __init__(self, in_features, out_features, bias=False):
        super().__init__(in_features, out_features, bias)
        self.input_quantizer = Ternary()
        self.weight_quantizer = Ternary()

    def forward(self, input_f):
        input_t = self.input_quantizer(input_f)
        weight_b = self.weight_quantizer(self.weight)
        out = F.linear(input_t, weight_b, self.bias)
        return out

In [422]:
class Net_tnn(nn.Module):
    def __init__(self):
        super(Net_tnn, self).__init__()

        self.linear1 = QLinear(13, 128, bias = False)
        # self.bn1 = nn.BatchNorm1d(128)
        # self.i1 = nn.Identity()

        self.linear2 = QLinear(128, 128, bias = False)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.i2 = nn.Identity()

        self.linear3 = QLinear(128, 128, bias = False)
        # self.bn3 = nn.BatchNorm1d(128)
        # self.i3 = nn.Identity()

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        # x = self.input_quantizer(x)
        x = self.linear1(x)
        # x = self.bn1(x)

        # x = self.input_quantizer(x)
        x = self.linear2(x)
        # x = self.bn2(x)

        # x = self.input_quantizer(x)
        x = self.linear3(x)
        # x = self.bn3(x)

        # x = self.input_quantizer(x)
        x = self.linear4(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_tnn = Net_tnn().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_tnn.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 4
for epoch in range(num_epochs):
    model_tnn.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_tnn(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_tnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_tnn(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_tnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_tnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_tnn.state_dict(), 'saved_best_model_tnn.pt')

Epoch [1/4], Validation Accuracy: 84.68%2
Epoch [2/4], Validation Accuracy: 91.85%8
Epoch [3/4], Validation Accuracy: 91.85%4
Epoch [4/4], Validation Accuracy: 91.85%9
Test Accuracy: 91.74%


In [655]:
def convert_and_concatenate_tnn(arr):
    arr = torch.flip(arr, dims=[0])
    # Chuyển đổi -1 thành 1 và 1 thành 0
    converted_bit0_arr = [1 if x == -1 else 0 for x in arr]
    converted_bit1_arr = [1 if x == 1 else 0 for x in arr]
    concatenated_bit0_str = ''.join(map(str, converted_bit0_arr))
    concatenated_bit1_str = ''.join(map(str, converted_bit1_arr))


    current_length = np.max([len(concatenated_bit0_str),len(concatenated_bit0_str)])
    total_length = (np.ceil(current_length/32)*32).astype(int)
    if current_length < total_length:
        padding_length0 = int(total_length - len(concatenated_bit0_str))
        padding_length1 = int(total_length - len(concatenated_bit1_str))
        concatenated_bit0_str = '0' * padding_length0 + concatenated_bit0_str
        concatenated_bit1_str = '0' * padding_length1 + concatenated_bit1_str

    # Chuyển chuỗi thành số nhị phân và sau đó thành số hex
    hex_value_bit0 = hex(int(concatenated_bit0_str, 2))[2:]  # Bỏ tiền tố '0x'
    hex_value_bit1 = hex(int(concatenated_bit1_str, 2))[2:]  # Bỏ tiền tố '0x'
    # Đảm bảo chuỗi hex đủ độ dài cần thiết
    if(len(hex_value_bit0)>len(hex_value_bit1)):
        hex_value_bit1 = '0' * (len(hex_value_bit0)-len(hex_value_bit1)) + hex_value_bit1
    elif(len(hex_value_bit0)<len(hex_value_bit1)):
        hex_value_bit0 = '0' * (len(hex_value_bit1)-len(hex_value_bit0)) + hex_value_bit0

    hex_length = total_length // 4  # 1 hex digit = 4 bits
    if len(hex_value_bit0) < hex_length:
        hex_value_bit0 = '0' * (hex_length - len(hex_value_bit0)) + hex_value_bit0
        hex_value_bit1 = '0' * (hex_length - len(hex_value_bit1)) + hex_value_bit1
    return hex_value_bit0, hex_value_bit1

def save_tnn_model_parameters_to_txt(model, file_path):
    input_thres = []
    weight_thres = []
    for name, module in model.named_modules():        
        if 'input_quantizer' in name:
            input_thres = np.append(input_thres, module.running_delta.item())
        if 'weight_quantizer' in name:
            weight_thres = np.append(weight_thres, module.running_delta.item())
    # input_thres = np.append(input_thres, 0)
    cnt=0
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f'{name}\n')
            f.write(f'input_channel: {param.shape[1]}\n')
            f.write(f'output_channel: {param.shape[0]}\n')
            f.write(f'input_thres: {input_thres[cnt]}\n')

            param = _ternary(param, weight_thres[cnt])
            print(name)
            for param_e in param:
                hex_conv_bit0, hex_conv_bit1 = convert_and_concatenate_tnn(param_e)
                segments0 = [hex_conv_bit0[i:i+8] for i in range(0, len(hex_conv_bit0), 8)]
                segments1 = [hex_conv_bit1[i:i+8] for i in range(0, len(hex_conv_bit1), 8)]
                segments0.reverse()
                segments1.reverse()
                for j in range (len(segments0)):
                    f.write(f'0x{segments0[j]}\n')
                    f.write(f'0x{segments1[j]}\n')
            f.write(f'\n')
            cnt+=1
    return 0

# Gọi hàm để lưu thông số mô hình
a = save_tnn_model_parameters_to_txt(model_tnn, 'tnn_model_parameters.txt')

linear1.weight
linear2.weight
linear3.weight
linear4.weight


In [649]:
class Net_tnn(nn.Module):
    def __init__(self):
        super(Net_tnn, self).__init__()

        self.linear1 = QLinear(13, 128, bias = False)
        # self.bn1 = nn.BatchNorm1d(128)
        # self.i1 = nn.Identity()

        self.linear2 = QLinear(128, 128, bias = False)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.i2 = nn.Identity()

        self.linear3 = QLinear(128, 128, bias = False)
        # self.bn3 = nn.BatchNorm1d(128)
        # self.i3 = nn.Identity()

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        # x = self.input_quantizer(x)
        x = self.linear1(x)
        print("linear1: ")
        print(x)

        # x = self.input_quantizer(x)
        x = self.linear2(x)
        print("linear2: ")
        print(x)
        # x = self.bn2(x)

        # x = self.input_quantizer(x)
        x = self.linear3(x)
        print("linear3: ")
        print(x)
        # x = self.bn3(x)

        # x = self.input_quantizer(x)
        x = self.linear4(x)
        print("linear4: ")
        print(x)
        x = self.act4(x)
        return x
    
model_tnn_test = Net_tnn().to(device)
checkpoint = torch.load('saved_best_model_tnn.pt')
model_tnn_test.load_state_dict(checkpoint)

<All keys matched successfully>

In [650]:
# Final evaluation on the test set
model_tnn_test.eval()
correct = 0
total = 0
test = torch.tensor([[0.0000,  0.0000,  0.0000,  0.0000, -0.6556,  0.9844,  0.2236, -1.4968, -1.4968, -0.9711, -0.1279,  0.2186,  0.2208]]).to(device)
with torch.no_grad():
    outputs = model_tnn_test(test)

# test_accuracy = 100 * correct / total
# print(f'Test Accuracy: {test_accuracy:.2f}%')

linear1: 
tensor([[-3.,  0.,  3.,  2., -4.,  0.,  1.,  1.,  0.,  1., -3., -1.,  0.,  1.,
          1.,  1., -3.,  1.,  1.,  1.,  3.,  0., -3.,  1.,  1., -1., -1.,  0.,
          0., -1., -1.,  1., -1.,  5., -2.,  1., -3., -2.,  1.,  1., -1.,  4.,
          1., -2., -1., -2.,  1.,  2.,  1., -2.,  1.,  0.,  0.,  1.,  1., -1.,
          0.,  2.,  0., -1.,  2.,  2.,  0., -1., -2., -3., -3., -3., -3.,  1.,
         -4.,  1., -3., -2., -1.,  1.,  0.,  3., -1.,  1.,  2.,  0., -2., -3.,
          0.,  2., -1.,  1., -2., -2.,  2.,  2., -1., -2.,  1.,  0.,  2.,  4.,
          5., -3., -2., -1.,  1., -1., -4.,  0.,  2., -2.,  1., -1.,  0., -1.,
          0.,  1.,  3.,  1.,  1.,  3.,  3., -1.,  1.,  2.,  3., -2.,  0.,  2.,
          0., -2.]], device='cuda:0')
linear2: 
tensor([[ 12.,   6., -10., -14.,  -5.,   8.,  10.,   1.,  31.,  12.,  10.,  -5.,
          -2.,  11., -12., -24.,  -1.,  -2.,  11.,  -4.,  12.,   4.,   2., -14.,
          -5.,  -2.,  -3.,   1.,   7., -15.,   5.,  -4.,  -4.,   9., 

0.3157

# **TBN**

In [ ]:
class QLinear(torch.nn.Linear):
    def __init__(self, in_features, out_features, bias=False):
        super().__init__(in_features, out_features, bias)
        self.input_quantizer = Ternary()
        self.weight_quantizer = BinaryActivation()

    def forward(self, input_f):
        input_t = self.input_quantizer(input_f)
        weight_b = self.weight_quantizer(self.weight)
        out = F.linear(input_t, weight_b, self.bias)
        return out

In [675]:
class Net_tbn(nn.Module):
    def __init__(self):
        super(Net_tbn, self).__init__()

        self.linear1 = QLinear(13, 128, bias = False)
        # self.bn1 = nn.BatchNorm1d(128)
        # self.i1 = nn.Identity()

        self.linear2 = QLinear(128, 128, bias = False)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.i2 = nn.Identity()

        self.linear3 = QLinear(128, 128, bias = False)
        # self.bn3 = nn.BatchNorm1d(128)
        # self.i3 = nn.Identity()

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        # x = self.input_quantizer(x)
        x = self.linear1(x)
        # x = self.bn1(x)

        # x = self.input_quantizer(x)
        x = self.linear2(x)
        # x = self.bn2(x)

        # x = self.input_quantizer(x)
        x = self.linear3(x)
        # x = self.bn3(x)

        # x = self.input_quantizer(x)
        x = self.linear4(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_tbn = Net_tbn().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_tbn.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 4
for epoch in range(num_epochs):
    model_tbn.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_tbn(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_tbn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_tbn(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_tbn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_tbn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_tbn.state_dict(), 'saved_best_model_tbn.pt')

Epoch [1/4], Validation Accuracy: 86.25%1
Epoch [2/4], Validation Accuracy: 88.99%8
Epoch [3/4], Validation Accuracy: 92.04%7
Epoch [4/4], Validation Accuracy: 92.04%3
Test Accuracy: 91.98%


In [682]:
def _binary(x: torch.Tensor):
    return (x >= 0).float() - (x < 0).float()

def convert_and_concatenate_tbn(arr):
    # Chuyển đổi -1 thành 1 và 1 thành 0
    arr = torch.flip(arr, dims=[0])
    converted_arr = [1 if x == -1 else 0 for x in arr]
    # Ghép các phần tử lại thành một chuỗi
    concatenated_str = ''.join(map(str, converted_arr))

    current_length = len(concatenated_str)
    total_length = (np.ceil(current_length/32)*32).astype(int)
    if current_length < total_length:
        padding_length = int(total_length - current_length)
        concatenated_str = '0' * padding_length + concatenated_str
    # Chuyển chuỗi thành số nhị phân và sau đó thành số hex
    hex_value = hex(int(concatenated_str, 2))[2:]  # Bỏ tiền tố '0x'

    # Đảm bảo chuỗi hex đủ độ dài cần thiết
    hex_length = total_length // 4  # 1 hex digit = 4 bits
    if len(hex_value) < hex_length:
        hex_value = '0' * (hex_length - len(hex_value)) + hex_value
    return hex_value

def save_tbn_model_parameters_to_txt(model, file_path):
    input_thres = []
    for name, module in model.named_modules():        
        if 'input_quantizer' in name:
            input_thres = np.append(input_thres, module.running_delta.item())
    # input_thres = np.append(input_thres, 0)
    cnt=0
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f'{name}\n')
            f.write(f'input_channel: {param.shape[1]}\n')
            f.write(f'output_channel: {param.shape[0]}\n')
            f.write(f'input_thres: {input_thres[cnt]}\n')

            param = _binary(param)
            print(name)
            for param_e in param:
                hex_conv_bit = convert_and_concatenate_tbn(param_e)
                segments = [hex_conv_bit[i:i+8] for i in range(0, len(hex_conv_bit), 8)]
                segments.reverse()
                for j in range (len(segments)):
                    f.write(f'0x{segments[j]}\n')
            f.write(f'\n')
            cnt+=1
    return 0

# Gọi hàm để lưu thông số mô hình
a = save_tbn_model_parameters_to_txt(model_tbn, 'tbn_model_parameters.txt')

linear1.weight
linear2.weight
linear3.weight
linear4.weight


In [684]:
class QLinear(torch.nn.Linear):
    def __init__(self, in_features, out_features, bias=False):
        super().__init__(in_features, out_features, bias)
        self.input_quantizer = Ternary()
        self.weight_quantizer = BinaryActivation()

    def forward(self, input_f):
        input_t = self.input_quantizer(input_f)
        weight_b = self.weight_quantizer(self.weight)
        out = F.linear(input_t, weight_b, self.bias)
        return out

class Net_tbn_test(nn.Module):
    def __init__(self):
        super(Net_tbn_test, self).__init__()

        self.linear1 = QLinear(13, 128, bias = False)
        # self.bn1 = nn.BatchNorm1d(128)
        # self.i1 = nn.Identity()

        self.linear2 = QLinear(128, 128, bias = False)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.i2 = nn.Identity()

        self.linear3 = QLinear(128, 128, bias = False)
        # self.bn3 = nn.BatchNorm1d(128)
        # self.i3 = nn.Identity()

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        # x = self.input_quantizer(x)
        x = self.linear1(x)
        print("linear1: ")
        print(x)

        # x = self.input_quantizer(x)
        x = self.linear2(x)
        print("linear2: ")
        print(x)
        # x = self.bn2(x)

        # x = self.input_quantizer(x)
        x = self.linear3(x)
        print("linear3: ")
        print(x)
        # x = self.bn3(x)

        # x = self.input_quantizer(x)
        x = self.linear4(x)
        print("linear4: ")
        print(x)
        x = self.act4(x)
        return x
    
model_tbn_test = Net_tbn_test().to(device)
checkpoint = torch.load('saved_best_model_tbn.pt')
model_tbn_test.load_state_dict(checkpoint)
model_tbn_test.linear1.state_dict()['weight'].copy_(_binary(model_tbn_test.linear1.state_dict()['weight']))
model_tbn_test.linear2.state_dict()['weight'].copy_(_binary(model_tbn_test.linear2.state_dict()['weight']))
model_tbn_test.linear3.state_dict()['weight'].copy_(_binary(model_tbn_test.linear3.state_dict()['weight']))
model_tbn_test.linear4.state_dict()['weight'].copy_(_binary(model_tbn_test.linear4.state_dict()['weight']))

tensor([[ 1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
          1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
          1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,
          1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
          1.,  1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,
         -1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
         -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
         -1., -1.],
        [-1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
          1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1.,
          1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,
         -1.,  1., -1., -1.,  1.

In [686]:
# Final evaluation on the test set
model_tbn_test.eval()
correct = 0
total = 0
test = torch.tensor([[0.0000,  0.0000,  0.0000,  0.0000, -0.6556,  0.9844,  0.2236, -1.4968, -1.4968, -0.9711, -0.1279,  0.2186,  0.2208]]).to(device)
with torch.no_grad():
    outputs = model_tbn_test(test)

# test_accuracy = 100 * correct / total
# print(f'Test Accuracy: {test_accuracy:.2f}%')

linear1: 
tensor([[-1., -3.,  1., -1.,  3., -1., -3.,  3.,  3.,  1.,  1., -3.,  3., -3.,
          3.,  1.,  1.,  3.,  3., -1., -1., -1., -1.,  1., -3.,  1.,  1.,  1.,
          1.,  1.,  1., -1.,  3.,  5., -5., -1.,  3.,  1., -1., -1.,  5., -5.,
          1., -5.,  1., -3., -1., -3.,  5.,  1., -1., -1., -1.,  1., -3., -5.,
          1., -3., -3., -1.,  5., -3., -5.,  1., -1.,  3., -1., -1., -3.,  3.,
         -1.,  5., -1., -1., -5., -1., -3.,  1.,  5., -1.,  5.,  3.,  3., -1.,
          1.,  3., -1., -3.,  1.,  5.,  5.,  1.,  3.,  3.,  1.,  3.,  1.,  3.,
         -1., -1.,  1., -3.,  3., -5., -1.,  1., -3., -1.,  1.,  1.,  5.,  1.,
         -3.,  5.,  3., -1., -5., -1., -3., -1.,  5.,  3., -1., -1.,  1.,  1.,
         -3.,  1.]], device='cuda:0')
linear2: 
tensor([[ -4., -28., -30.,  -2.,   4.,  48.,  -2., -14.,  -4., -16.,   8.,  16.,
           6.,   4.,   8.,  28.,  -8., -16.,   6.,  -6.,  20.,  26.,   4., -30.,
          20., -10., -16.,   6.,   4., -42.,  18., -34., -10., -12., 

# **GENERATE TESTCASE**

In [752]:
df_combined = pd.concat([df_bot_dos, df_bot_ddos], ignore_index=True)

x = df_combined.loc[:, df_combined.columns != 'category']
# x = (x-x.min())/(x.max()-x.min())

df_combined.category = pd.factorize(df_combined.category)[0]
y = df_combined['category']

scaler = StandardScaler()
x = scaler.fit_transform(x)
for i in [0,1,2,3,7,8]:
    x[:,i] = np.where(x[:,i] > 0, np.log(x[:,i]), 0)
    
x_label_data = torch.tensor(x, dtype=torch.float).to(device)
y_label_data = torch.tensor(y, dtype=torch.long).to(device)

dataset = TensorDataset(x_label_data, y_label_data)
train_size = int(0.7 * len(dataset))
valid_size = int(0.275 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# Create data loaders
batch_size = 8192
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

/tmp/ipykernel_12712/3797725696.py:12: RuntimeWarning: invalid value encountered in log
  x[:,i] = np.where(x[:,i] > 0, np.log(x[:,i]), 0)


In [753]:
test_size

84000

In [ ]:
def save_tbn_model_parameters_to_txt(model, file_path):
    input_thres = []
    for name, module in model.named_modules():        
        if 'input_quantizer' in name:
            input_thres = np.append(input_thres, module.running_delta.item())
    # input_thres = np.append(input_thres, 0)
    cnt=0
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f'{name}\n')
            f.write(f'input_channel: {param.shape[1]}\n')
            f.write(f'output_channel: {param.shape[0]}\n')
            f.write(f'input_thres: {input_thres[cnt]}\n')

            param = _binary(param)
            print(name)
            for param_e in param:
                hex_conv_bit = convert_and_concatenate_tbn(param_e)
                segments = [hex_conv_bit[i:i+8] for i in range(0, len(hex_conv_bit), 8)]
                segments.reverse()
                for j in range (len(segments)):
                    f.write(f'0x{segments[j]}\n')
            f.write(f'\n')
            cnt+=1
    return 0

In [754]:

tc = 0


In [ ]:
file_path = 'testcase.txt'
with open(file_path, 'w') as f:
    for images, labels in test_loader:
        f.write(f'testcase: {tc}\n')
        f.write(f'data: {images[0]}\n'.replace(" ", ""))
        f.write(f'label: {labels[0].cpu().numpy()}\n')
        f.write(f'\n')
        tc +=1
